In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal


import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
from common import train
os.environ['http_proxy'] = "http://192.41.170.23:3128" 
os.environ['https_proxy'] = "http://192.41.170.23:3128" 

In [2]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

'NVIDIA RTX A6000'

In [3]:
from common import EEG_fif
from common import create_dataloader

In [4]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, out_size):
        super().__init__()
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        
        
        self.fc = nn.Linear(linear_shape  , out_size)
        #self.fc = nn.Linear(80064 , 2)
        
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = self.layer3(out)
        #out = self.drop_out(out)
        #out = self.layer4(out)
        #out = self.drop_out(out)

        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [8]:
subjects = [20]
print("subjects number : {} ".format(subjects))

#[20,31,47,48,49,54,55,59,62]# list(range(36, 63))

in_channels  = 2 #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5



path = ''
base_url = 'https://physionet.org/files/eegmmidb/'



subjects number : [20] 


In [9]:
for i in range(3):
    if i == 0:
        n_type = "EX"
    if i == 1:
        n_type = "MI"
    if i == 2:
        n_type = "ALL"

    if n_type == "EX":
        runs = [3,5,7,9]
    if n_type == "MI":
        runs = [4,6,8,10]
    if n_type == "ALL":
        runs = [3,5,7,9,4,6,8,10]
    for subject in subjects:
        print("############## {} ########".format(subject))


        eeg = EEG_fif(path, base_url, [subject], runs)
        raw=eeg.data_to_raw()
        print("Raw done")
        raw=raw.notch_filter([50])
        raw=raw.filter( 8,14, method='fir', verbose=20)
        #raw.resample(160)
        print("Filter done")
        raw.pick_channels(['C3', 'C4', 'STIM MARKERS'])
        epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,1))
        X, y = eeg.get_X_y(epochs)
        y=y-1
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

        print('train size',X_train.shape, y_train.shape)
        print('Test size',X_test.shape, y_test.shape)
        batch_size = X_train.shape[2]
        train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
        test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
        num_step =math.ceil(len(train_loader.dataset) / batch_size)

        x_train = torch.FloatTensor(X_train)
        fake_eeg_data = torch.randn_like(x_train) #(batch_size, channel, length)

            #using sequential helps bind multiple operations together
        layer1 = nn.Sequential(
                    #padding = (kernel_size - 1) / 2 = 2
                    nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer2 = nn.Sequential(
                    nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels*2),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer3 = nn.Sequential(
                    nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                )


        out = layer1(fake_eeg_data)
        out = layer2(out)
        out = layer3(out)
        out = out.reshape(out.size(0), -1)
        linear_shape = out.shape[1]
        fc = nn.Linear(linear_shape, out_size)
        out = fc(out)

        wand = wandb.init(

              # Set the project where this run will be logged
              project="Motor-Imagery", 
              # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
              name="CNN_3layer_2class_S{0}_{1}_X".format(subject,n_type), 
              # Track hyperparameters and run metadata
              config={
              "learning_rate": 0.00001,
              "architecture": "CNN",
              "dataset": "S{}".format(subjects[0]),
              "epochs": 3000,
              "weightname":"CNN_3layer_2class_S{0}_{1}_X".format(subject,n_type),
              "num_step_per_epoch" : num_step, 

              }
            )
        wandb.login()
        config = wand.config
        net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
        optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
        criterion = nn.CrossEntropyLoss()


        train_loss,valid_loss,train_accuracy,valid_accuracy =train(
            model = net,
            gpu_num = 0,
            train_loader = train_loader,
            test_loader = test_loader,
            optimizer = optimizer  ,
            criterion = criterion ,
            wand = wand
                 )


        wandb.alert(
                    title='Finish',
                    text=f'Finishing training',
                )

############## 20 ########

Raw done
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method


/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S020/S020R03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S020/S020R05.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/home/nutapolt/eeg_mi/common.py:627: RuntimeWarning: This filename (S020/S020R07.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fi

- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 sec)

Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filter done
360 events found
Event IDs: [1 2 4]
Not setting metadata
120 matching events found
Applying baselin

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/nutapolt/eeg_mi/common.py:657: RuntimeWarning: No ma

train size (81, 2, 1751) (81,)
Test size (35, 2, 1751) (35,)


...

Filtering raw data in 8 contiguous segments
Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filter done
720 events found
Event IDs: [1 2 4]
Not setting metadata
240 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 240 events and 1751 original time points ...
    Rejecting  epoch based on EEG : ['C4']
9 bad epochs dropped
train size (161, 2, 1751) (161,)
Test size (70, 2, 1751) (70,)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/nutapolt/eeg_mi/common.py:657: RuntimeWarning: No matching events found for 3 (event id 3)
  epochs = mne.Epochs(


Test/Test_accuracy,▅▅▅▅▅▅▇▆▇██▅▂▆█▄▄▁█▄▂▂▃▆▄▃▄▄▄▄▆▃▄▄▅▄▄▄▅▄
Test/Test_loss,▁▅█▅▅▅▂▃▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▃▃▂▂
train/train_accuracy,▁███████████████████████████████████████
train/train_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test/Test_accuracy,45.71429
Test/Test_loss,1.50196
train/train_accuracy,100.0
train/train_loss,0.00024


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Epoch 1/3000, Tr Loss: 0.6839, Tr Acc: 58.3851, Val Loss: 0.6935, Val Acc: 48.5714
Epoch 101/3000, Tr Loss: 0.1962, Tr Acc: 98.1367, Val Loss: 2.6079, Val Acc: 48.5714
Epoch 201/3000, Tr Loss: 0.0673, Tr Acc: 100.0000, Val Loss: 16.4396, Val Acc: 48.5714
Epoch 301/3000, Tr Loss: 0.0269, Tr Acc: 100.0000, Val Loss: 3.3246, Val Acc: 48.5714
Epoch 401/3000, Tr Loss: 0.0163, Tr Acc: 100.0000, Val Loss: 6.8935, Val Acc: 48.5714
Epoch 501/3000, Tr Loss: 0.0119, Tr Acc: 100.0000, Val Loss: 0.5581, Val Acc: 72.8571
Epoch 601/3000, Tr Loss: 0.0085, Tr Acc: 100.0000, Val Loss: 0.8388, Val Acc: 65.7143
Epoch 701/3000, Tr Loss: 0.0055, Tr Acc: 100.0000, Val Loss: 1.3919, Val Acc: 62.8571
Epoch 801/3000, Tr Loss: 0.0039, Tr Acc: 100.0000, Val Loss: 0.6477, Val Acc: 81.4286
Epoch 901/3000, Tr Loss: 0.0036, Tr Acc: 100.0000, Val Loss: 0.6456, Val Acc: 75.7143
Epoch 1001/3000, Tr Loss: 0.0027, Tr Acc: 100.0000, Val Loss: 0.6293, Val Acc: 80.0000
Epoch 1101/3000, Tr Loss: 0.0023, Tr Acc: 100.0000, Val 

In [ ]:
##PACK
for i in range(3):
    if i == 0:
        n_type = "EX"
    if i == 1:
        n_type = "MI"
    if i == 2:
        n_type = "ALL"

    if n_type == "EX":
        runs = [3,5,7,9]
    if n_type == "MI":
        runs = [4,6,8,10]
    if n_type == "ALL":
        runs = [3,5,7,9,4,6,8,10]
    print("############## {} ########".format(subjects))

    eeg = EEG_fif(path, base_url, subjects, runs)
    raw=eeg.data_to_raw()
    print("Raw done")
    raw=raw.notch_filter([50])
    raw=raw.filter( 8,14, method='fir', verbose=20)
    #raw.resample(160)
    print("Filter done")
    raw.pick_channels(['C3', 'C4', 'STIM MARKERS'])
    epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,1))
    X, y = eeg.get_X_y(epochs)
    y=y-1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

    print('train size',X_train.shape, y_train.shape)
    print('Test size',X_test.shape, y_test.shape)
    batch_size = X_train.shape[2]
    train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
    test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
    num_step =math.ceil(len(train_loader.dataset) / batch_size)

    x_train = torch.FloatTensor(X_train)
    fake_eeg_data = torch.randn_like(x_train) #(batch_size, channel, length)

        #using sequential helps bind multiple operations together
    layer1 = nn.Sequential(
                #padding = (kernel_size - 1) / 2 = 2
                nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer2 = nn.Sequential(
                nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels*2),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer3 = nn.Sequential(
                nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
            )


    out = layer1(fake_eeg_data)
    out = layer2(out)
    out = layer3(out)
    out = out.reshape(out.size(0), -1)
    linear_shape = out.shape[1]
    fc = nn.Linear(linear_shape, out_size)
    out = fc(out)

    wand = wandb.init(

          # Set the project where this run will be logged
          project="Motor-Imagery", 
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name="CNN_3layer_2class_S{0}_{1}_ne".format(subjects,n_type), 
          # Track hyperparameters and run metadata
          config={
          "learning_rate": 0.000001,
          "architecture": "CNN",
          "dataset": "S{}".format(subjects),
          "epochs": 3000,
          "weightname":"CNN_3layer_2class_S{0}_{1}_ne".format(subjects,n_type),
          "num_step_per_epoch" : num_step, 

          }
        )
    wandb.login()
    config = wand.config
    net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
    optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss()


    train_loss,valid_loss,train_accuracy,valid_accuracy =train(
        model = net,
        gpu_num = 0,
        train_loader = train_loader,
        test_loader = test_loader,
        optimizer = optimizer  ,
        criterion = criterion ,
        wand = wand
             )


    wandb.alert(
                title='Finish',
                text=f'Finishing training',
            )